In [1]:
# import subprocess

In [2]:
# http://pytorch.org/
try:
    import torch
except ImportError:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
    
    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl
    # !pip install torchvision
    # subprocess.check_call(['pip', 'install', '-q', f'http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl'])
    import torch

In [3]:
try:
    import torchtext
except ImportError:
    # subprocess.check_call(['pip', 'install', 'torchtext'])
    !pip install torchtext
try:
    import spacy
except ImportError:
    # subprocess.check_call(['pip', 'install', 'spacy'])
    !pip install spacy

In [4]:
import torch 
from torchtext import data
torch.__version__

'0.4.1'

In [5]:
import spacy.cli
spacy.cli.download('en')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [6]:
import torch
from torchtext import data
from torchtext import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [7]:
print(len(train_data))
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train_data)

17500


In [8]:
print(len(test_data))

25000


In [9]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, 
    device=device)

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs,embedding_dim)) for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)


In [11]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [12]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.1708, -0.3717,  0.1077,  ...,  0.4888,  0.3385,  0.0938],
        [-0.4776,  0.6565,  0.4362,  ..., -0.9675,  0.5557, -0.1936],
        [-0.2001,  0.0097, -0.7372,  ...,  0.0367,  0.6546,  0.4138]])

In [13]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [14]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [15]:

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
       
        predictions = model(batch.text).squeeze(1)

        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

| Epoch: 01 | Train Loss: 0.503 | Train Acc: 74.21% | Val. Loss: 0.353 | Val. Acc: 84.63% |
| Epoch: 02 | Train Loss: 0.307 | Train Acc: 86.92% | Val. Loss: 0.334 | Val. Acc: 85.72% |
| Epoch: 03 | Train Loss: 0.221 | Train Acc: 91.14% | Val. Loss: 0.271 | Val. Acc: 88.87% |
| Epoch: 04 | Train Loss: 0.147 | Train Acc: 94.46% | Val. Loss: 0.291 | Val. Acc: 88.54% |
| Epoch: 05 | Train Loss: 0.091 | Train Acc: 96.91% | Val. Loss: 0.291 | Val. Acc: 89.17% |


In [18]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

| Test Loss: 0.312 | Test Acc: 88.34% |


In [19]:
torch.save(model.state_dict(),'model_cnn.pth')

In [20]:
dick = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)
dick.load_state_dict(torch.load('model_cnn.pth'))
dick = dick.to(device)

In [21]:
import time
start_time = time.time()
test_loss, test_acc = evaluate(dick, test_iterator, criterion)
end_time = time.time()

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')
print(end_time-start_time)

| Test Loss: 0.312 | Test Acc: 88.34% |
6.039096117019653


In [22]:
# TREC
TREC_TEXT = data.Field(tokenize='spacy')
TREC_LABEL = data.LabelField(dtype=torch.float)

TREC_train_data, TREC_test_data = datasets.TREC.splits(TREC_TEXT, TREC_LABEL)

TREC_train_data, TREC_valid_data = TREC_train_data.split(random_state=random.seed(SEED))

In [23]:
print(len(TREC_train_data))
print(len(TREC_valid_data))
print(len(TREC_test_data))
TREC_TEXT.build_vocab(TREC_train_data, max_size=25000, vectors="glove.6B.100d")
TREC_LABEL.build_vocab(TREC_train_data)

3816
1636
500


In [24]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TREC_train_iterator, TREC_valid_iterator, TREC_test_iterator = data.BucketIterator.splits(
    (TREC_train_data, TREC_valid_data, TREC_test_data), 
    batch_size=BATCH_SIZE, 
    device=device)

In [25]:
INPUT_DIM = len(TREC_TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 6
DROPOUT = 0.5

TREC_model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [26]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.NLLLoss()

TREC_model = TREC_model.to(device)
criterion = criterion.to(device)

In [27]:
import numpy as np
def multiclass_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = np.argmax(nn.Softmax(preds),axis = 0)
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [28]:
def make_one_hot(labels):
    '''
    Converts an integer label Tensor/Variable to a one-hot variable.
    
    Parameters
    ----------
    labels : torch.Tensor or torch.Variable
        N x H x W, where N is batch size. 
        Each value is an integer representing correct classification.
    
    Returns
    -------
    target : torch.LongTensor or torch.Variable
        N x C x H x W, where C is class number. One-hot encoded.
        Returns `Variable` if `Variable` is given as input, otherwise
        returns `torch.LongTensor`.
    '''
    v = False
    if isinstance(labels, torch.autograd.variable.Variable):
        labels = labels.data
        v = True
        
    C = labels.max() + 1
    labels_ = labels.unsqueeze(1)
    one_hot = torch.LongTensor(labels_.size(0), C, labels_.size(2), labels_.size(3)).zero_()
    target = one_hot.scatter_(1, labels_, 1)
    
    if v and torch.cuda.is_available():
        target = Variable(target)
        
    return target

In [29]:
for batch in TREC_train_iterator:
        batch.label = batch.label.reshape(1,1,batch.label.shape[0])
        batch.label= batch.label.permute(0,2,1) 
        batch_size, k, _ = batch.label.size()
        labels_one_hot = torch.cuda.FloatTensor(batch_size, k, 6).zero_()
        
        labels_one_hot.scatter_(2, batch.label.long(), 1)
        print(labels_one_hot)
  

tensor([[[0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 0., 1.

tensor([[[0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 1.

tensor([[[0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 0., 0.

In [30]:
def one_hot(label,num_classes):
#         print(batch.label.shape[0])
        label = label.reshape(1,1,label.shape[0])
        label=label.permute(0,2,1) 
        batch_size, k, _ = label.size()
        labels_one_hot = torch.cuda.LongTensor(batch_size, k, num_classes).zero_()
        
        labels_one_hot.scatter_(2, label.long(), 1)
        return labels_one_hot.squeeze(0)
#         print(labels_one_hot)


In [31]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
       
        predictions = model(batch.text).squeeze(1)
        print(type(predictions))
        print(predictions.shape)
        print(type(one_hot(batch.label,6)))
        print(one_hot(batch.label,6).shape)
        loss = criterion(predictions, batch.label.long())
        
        acc = multiclass_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions,batch.label.long())
            
            acc = multiclass_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [33]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    TREC_train_loss, TREC_train_acc = train(TREC_model, TREC_train_iterator, optimizer, criterion)
    TREC_valid_loss, TREC_valid_acc = evaluate(TREC_model, TREC_valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'tor

<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'tor

<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([40, 6])
<class 'torch.Tensor'>
torch.Size([40, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'tor

<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
<class 'torch.Tensor'>
torch.Size([64, 6])
| Epoch: 05 | Train Loss: 0.091 | Train Acc: 96.91% | Val. Loss: 0.291 | Val. Acc: 89.17% |
